# This is a script for downloading and processing GTRD databases with ChIP-seq data for different organisms 

#### &copy; Wanwan Ge, 2018-05-12

## First, load python libraries

In [1]:
import pandas as pd
import os
import glob
import re
import numpy as np
import yaml
import requests   # download files from url
import gzip
import shutil
import subprocess # run bash in python

## Second, define functions
#### a) Split raw downloaded meta file into .bed files for each TF

In [2]:
# split mata file into .bed file for every individuel TF
def split_meta_table_to_bed(ifile, hm_dir):
    
    odir = os.path.join(hm_dir, 'bed_from_metacluster/')
    if not os.path.exists(odir):
        os.makedirs(odir)

    meta_file = pd.read_csv(ifile, header=0, sep='\t')

    grouped = meta_file.groupby(meta_file['tfTitle'])

    for name, group in grouped:
        TFname = name.split(' (')[0]
        TFname = TFname.split(' [')[0]
        TFname = TFname.replace('&#945;', '-alpha')
        TFname = TFname.replace('&#946;', '-beta')
        TFname = TFname.replace('&#947;', '-gamma')
        TFname = TFname.replace('&#948;', '-delta')
        TFname = TFname.replace('&#949;', '-epsilon')
        TFname = TFname.replace('&#954;', '-kappa')
        TFname = TFname.replace('/', '_')
        TFname = TFname.replace(' ', '-')
        TFname = TFname.replace('--', '-')
        ofile = os.path.join(odir, TFname+'.bed')
        group.to_csv( ofile, header=True, index=None, sep='\t', mode='w')

#### b) Modify .bed files so that only max. 5000 sequences of length 200nt will be chosen, after sorting by ranking scores

In [3]:
# extract sequences from the summit with a window size of 201 nt around the summit
# rerank the peaks by peak count before extracting
# extract maximum 5000 sequences
def extract_bed_from_bed(indir, hm_dir, summit=100, maxN=5000):
    
    odir_bed = os.path.join(hm_dir, 'bed_summit100/')
    odir_meta = os.path.join(hm_dir, 'meta_individual/')
    
    if not os.path.exists(odir_bed):
        os.makedirs(odir_bed)

    if not os.path.exists(odir_meta):
        os.makedirs(odir_meta)

    for i, filename in enumerate( sorted( glob.glob( os.path.join(indir +'*.bed')) ), start=1 ):
        filename = os.path.basename(filename)
        basename = os.path.splitext(filename)[0]
        table = pd.read_csv(os.path.join(indir+filename), header=0, sep='\t').values
        seqname = table[:,0]
        chromstart = table[:,1]
        peak = table[:,3]
        qVal = table[:,12] # peak.count
        seqstart = chromstart + peak - summit
        seqend = chromstart + peak + summit
        # note: how to deal with seqstart < 0?
        
        qVal_sorted = qVal.argsort() # rerank sequences due to peak counts
        
        seqname = seqname[qVal_sorted[::-1]]
        seqstart = seqstart[qVal_sorted[::-1]]
        seqend = seqend[qVal_sorted[::-1]]
        qVal = qVal[qVal_sorted[::-1]]
        opath = os.path.join(odir_bed, basename + '_summits'+str(summit)+'.bed')
        ofile = open(opath, "w")
        length = min(maxN, len(seqname))
        ofile.write('#CHROM\tSTART\tEND\tqVal\n')
        for i in range(length):
            ofile.write(seqname[i]+'\t'+str(seqstart[i])+'\t'+str(seqend[i])+'\t'+str(qVal[i])+'\n')
        ofile.close()
        
        # generate meta_file for each individual dataset
        opath_meta = os.path.join(odir_meta, basename + '.meta')
        ofile_meta = open(opath_meta, "w")
        ofile_meta.write('target_name\ttfClassId.uniprotId\ttfTitle\tcell_set\ttreatment_set\texp_set\n')
        ofile_meta.write(basename+'\t'
                         +str(table[:,4][0])+'\t'
                         +str(table[:,5][0])+'\t'
                         +str(table[:,6][0])+'\t'
                         +str(table[:,7][0])+'\t'
                         +str(table[:,8][0])+'\n')
        ofile_meta.close()

#### c) Generate motifs.yaml from original meta file for summarizing motif information

In [29]:
# generate motifs.yaml from meta file
def create_yaml_from_meta_bed(idir_meta, db_dir, species):
    count = 0
    items = []
    ofile_yaml = db_dir+'/motifs.yaml'
    for path2file in glob.glob(idir_meta+'/*'):
        count += 1
        TF = os.path.basename(path2file).split('.meta')[0]
        file = pd.read_csv( path2file, sep='\t', header=0 )
        item = {
            'filename': TF,
            'target_name': TF,
            'motif_id': 'GTRD_'+species+'_v101_'+str(count).zfill(4),
            'pos_seq_file': 'source/sequences/'+TF+'.fasta',
            'motif_init_file': 'source/initPWMs/'+TF+'.meme',
            'TF_class_id': file['tfClassId.uniprotId'][0],
            'TF_title': file['tfTitle'][0],
            'cell_type': file['cell_set'][0]
        }
        items.append(item)

    total = {
        'models': items
    }

    ofile = open(ofile_yaml, 'w')

    ofile.write( yaml.dump(total, default_flow_style=False) )

    ofile.close()

#### d) generate YAML files for each database

In [8]:
def create_yamls_for_DB(opath_db, species, fullname):
    # 1. generate database_config.yaml
    ofile_config = opath_db+'/database_config.yaml'
    with open(ofile_config, "w") as fh:
        print("version: 1.0.1", file=fh)
        print("display_name: GTRD-"+species.upper(), file=fh)
        print("organism: "+fullname, file=fh)

    # 2. generate model_specifications.yaml
    ofile_spec = opath_db+'/model_specifications.yaml'
    with open(ofile_spec, "w") as fh:
        print("model_specification: ", file=fh)
        print("  param_id: ", file=fh)
        print("  data_source: GTRD", file=fh)
        print("  species: "+fullname, file=fh)
        print("  experiment: ChIPseq", file=fh)
        print("  motif_init_file_format: fasta", file=fh)
        print("  reversecomp: True", file=fh)
        print("  modelorder: 4", file=fh)
        print("  extend_1: 0", file=fh)
        print("  extend_2: 0", file=fh)
        print("  bgmodelorder: 2", file=fh)
        print("  em: True", file=fh)
        print("  fdr: True", file=fh)
        print("  mfold: 1", file=fh)
        print("  samplingorder: 2", file=fh)

#### e) download meta cluster file from GTRD url, given species name and home directory

In [16]:
def download_metacluster(url, species, hm_dir, species):
    
    ofile_meta = os.path.join(hm_dir, species+'_meta_clusters.interval')
    
    # download meta clusters from the url
    ofile_gz = ofile_meta+'.gz'
    response = requests.get(url)
    with open(ofile_gz, 'wb') as f:
        f.write(response.content)

    # unzip file
    with gzip.open(ofile_gz, 'rb') as f_in:
        with open(ofile_meta, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)


#### f) create empty folders for hosting initial source and optimized models 

In [22]:
def create_db_folders(db_dir):
    # create empty folders
    odir_source = os.path.join(db_dir, 'source')
    odir_models = os.path.join(db_dir, 'models')
    if not os.path.exists(odir_source):
        os.mkdir(odir_source)
    if not os.path.exists(odir_models):
        os.mkdir(odir_models)
    odir_seqs = os.path.join(odir_source, 'sequences')
    odir_pwms = os.path.join(odir_source, 'initPWMs')
    if not os.path.exists(odir_seqs):
        os.mkdir(odir_seqs)
    if not os.path.exists(odir_pwms):
        os.mkdir(odir_pwms)

## Main Function
### Here is an example of processing human database
#### 1) Define pathes and variables

In [25]:
gtrd_dir = '/home/wanwan/benchmark/GTRD/'
serverdir = '/home/wanwan/workspace/webserver/motif_db/'

species = 'human'
fullname = 'Homo sapiens'
url = 'http://gtrd.biouml.org/downloads/18.01/human_meta_clusters.interval.gz'

hm_dir = gtrd_dir + species
if not os.path.exists(hm_dir):
    os.mkdir(hm_dir)
db_dir = os.path.join(serverdir, 'GTRD_v1804_'+species.upper())
if not os.path.exists(db_dir):
    os.mkdir(db_dir)
    
meta_dir_filtered = os.path.join(hm_dir, 'meta_individual_filtered/')

#### 2) download meta clusters from GTRD website: http://gtrd.biouml.org/

In [13]:
download_metacluster(url, species, hm_dir, species) # This takes a while

#### 3) split meta clusters into bed files for each TF

In [19]:
split_meta_table_to_bed(ofile_meta, hm_dir) # This takes a while

/home/wanwan/anaconda3/envs/sci35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [21]:
# convert bed file to another bed file with window size of 201 from the summit
extract_bed_from_bed(original_bed_dir, hm_dir, meta_dir)

#### Note: in order to extract .fasta file of sequences from the bed file, you need to download the reference genomes
##### http://hgdownload.soe.ucsc.edu/downloads.html

#### 4) download reference genome for this species locally

#### 5) Extract FASTA file from BED file using bedtools
###### Run it in shell

In [ ]:
%%bash
dir=/home/wanwan/benchmark/GTRD/yeast
mkdir -p ${dir}/fasta_summit100
# make sure there is one and only one reference genome file (.fa) in the dir
for file in ${dir}/bed_summit100/*.bed
do 
    bedfile=$(basename ${file})
    TFname=${bedfile%_summits100.*}
    bedtools getfasta -fi ${dir}/*.fa -bed ${file} -fo ${dir}/fasta_summit100/${TFname}.fasta
done 

#### 6) Run PEnGmotif and BaMMmotif for discovering and optimizating motifs
##### Here is an example for running it locally.
##### Note: it is better to run it on the cluster. Please copy the FASTA folder and the corresponding bash script to the cluster

In [ ]:
%%bash
DIR=/home/wanwan/benchmark/GTRD/yeast
FASTA_DIR=${DIR}/fasta_summit100
peng_dir=${DIR}/initPWMs
bamm_dir=${DIR}/models
mkdir -p ${peng_dir}
mkdir -p ${bamm_dir}
for file in ${FASTA_DIR}/*.fasta
do
    fastafile=$(basename ${file})
    bn=${fastafile%.fasta}
    shoot_peng.py -o ${peng_dir}/${bn}.meme -d ${peng_dir}/${bn}_tmp -w 8 ${FASTA_DIR}/${bn}.fasta > ${peng_dir}/${bn}.shootpeng.logg
    rm -fr ${peng_dir}/${bn}_tmp
    BaMMmotif ${bamm_dir}/${bn} ${FASTA_DIR}/${bn}.fasta --PWMFile ${peng_dir}/${bn}.meme --maxPWM 3 --EM -k 4 -q 0.3 --FDR -m 1 --scoreSeqset --extend 2 2 > ${bamm_dir}/${bn}.bamm.logg
    plotPvalStats.R ${bamm_dir}/${bn}/ ${bn} --plots 1
    plotMotifDistribution.R ${bamm_dir}/${bn}/ ${bn}
    plotBaMMLogo.R ${bamm_dir}/${bn}/ ${bn} 0 --web 1
    plotBaMMLogo.R ${bamm_dir}/${bn}/ ${bn} 1
    plotBaMMLogo.R ${bamm_dir}/${bn}/ ${bn} 2 
done 

#### 7) Create empty folders for hosting the initial source and optimized models

In [23]:
create_db_folders(db_dir)

#### 8) Copying optimized motifs from cluster to local
#### 9) Filtering step: filter the collection of meta files according to the actural TFs with motifs discovered. 
This is due to the reason that some TFs have too few sequences to search for motifs.

In [26]:
%%bash
meta_dir=/home/wanwan/benchmark/GTRD/human/meta_individual
meta_filter_dir=/home/wanwan/benchmark/GTRD/human/meta_individual_filtered
mkdir -p ${meta_filter_dir}
peng_dir=/home/wanwan/workspace/webserver/motif_db/GTRD_v1804_HUMAN/source/initPWMs
for file in ${peng_dir}/*.meme
do
    memefile=$(basename ${file})
    bn=${memefile%.meme}
    cp -r ${meta_dir}/${bn}.meta ${meta_filter_dir}/
done

#### 10) Create motifs.yaml file

In [30]:
create_yaml_from_meta_bed(meta_dir_filtered, db_dir, species)

#### 11) Create two other YAML files for the database

In [28]:
create_yamls_for_DB(db_dir, species, fullname)

#### 12) Zipping all the files for each individual TF

In [ ]:
%%bash
# zip files; -j flag is important for ignoring the path
dir=/home/wanwan/workspace/webserver/motif_db/GTRD_v1804_HUMAN
peng_dir=${dir}/source/initPWMs
model_dir=${dir}/models
for file in ${peng_dir}/*.meme
do
    memefile=$(basename ${file})
    bn=${memefile%.meme}
    zip -j ${model_dir}/${bn}/${bn}_complete.zip ${model_dir}/${bn}/*
done

#### 13) Last step: zip the whole database folder for uploading 

## Repeat the same steps for other databases

### Additional information:


#### For mouse:

In [32]:
species = 'mouse'
fullname = 'Mus musculus'
url = 'http://gtrd.biouml.org/downloads/18.01/mouse_meta_clusters.interval.gz'

#### For yeast database:

In [31]:
species='yeast'
fullname='Schizosaccharomyces pombe'
url='http://gtrd.biouml.org/downloads/18.06alpha/Schizosaccharomyces_pombe_meta_clusters.interval.gz'

#### etc...